## Milestone 2 steps:

### - Load .jp2 files
### - Add multiple bands in field list
### - Add attribute dataset with filename of all .jp2 within directory
### - Resample queried field dataset according to loaded dataset


In [1]:
import rasterio
from rasterio.mask import mask
from rasterio.windows import from_bounds
from rasterstats import zonal_stats
from rasterio.windows import Window
from shapely.geometry import box
from pathlib import Path
import numpy as np
from scipy.ndimage import label, generate_binary_structure
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import contextily as ctx
import time

# yt packages
import yt
import yt.extensions.geotiff

# CPU usage
import psutil

# package of resampling methods
from rasterio.enums import Resampling

from rasterio.warp import calculate_default_transform, reproject, Resampling


## Load *.jp2 format and add field bands

In [2]:
s2_file = ('C:/Users/arevi/OneDrive/YT_GITHUB_v2/TEST_DATASETS/s2_sample/T30UVG_20200601T113331_B02_10m.jp2')

In [3]:
ds = yt.load(s2_file)

yt : [INFO     ] 2021-03-16 14:49:27,100 Parameters: domain_dimensions         = [10980 10980     1]
yt : [INFO     ] 2021-03-16 14:49:27,100 Parameters: domain_left_edge          = [ 399960. 6090240.       0.] m
yt : [INFO     ] 2021-03-16 14:49:27,100 Parameters: domain_right_edge         = [5.09760e+05 6.20004e+06 1.00000e+00] m


In [4]:
ds.field_list

[('bands', 'AOT_10'),
 ('bands', 'AOT_20'),
 ('bands', 'AOT_60'),
 ('bands', 'B01_60'),
 ('bands', 'B02_10'),
 ('bands', 'B02_20'),
 ('bands', 'B02_60'),
 ('bands', 'B03_10'),
 ('bands', 'B03_20'),
 ('bands', 'B03_60'),
 ('bands', 'B04_10'),
 ('bands', 'B04_20'),
 ('bands', 'B04_60'),
 ('bands', 'B05_20'),
 ('bands', 'B05_60'),
 ('bands', 'B06_20'),
 ('bands', 'B06_60'),
 ('bands', 'B07_20'),
 ('bands', 'B07_60'),
 ('bands', 'B08_10'),
 ('bands', 'B09_60'),
 ('bands', 'B11_20'),
 ('bands', 'B11_60'),
 ('bands', 'B12_20'),
 ('bands', 'B12_60'),
 ('bands', 'B8A_20'),
 ('bands', 'B8A_60'),
 ('bands', 'SCL_20'),
 ('bands', 'SCL_60'),
 ('bands', 'TCI_10'),
 ('bands', 'TCI_20'),
 ('bands', 'TCI_60'),
 ('bands', 'WVP_10'),
 ('bands', 'WVP_20'),
 ('bands', 'WVP_60')]

In [ ]:
ds._field_filename

In [5]:
# Define dimensions
width = ds.arr(1000, 'm')
height = ds.arr(1000,'m') 
rectangle_centre = ds.arr([487309,6196162],'m')

rectangular_yt_container = ds.rectangle_from_center(rectangle_centre,width,height)

In [6]:
rectangular_yt_container[('bands','B02_60')]

Resampling B02_60


unyt_array([373., 373., 373., ..., 370., 493., 493.], '(dimensionless)')

In [7]:
p = ds.plot(('bands', 'B04_20'), height=height, width=width, center=rectangle_centre)
p.set_log(('bands', 'B04_20'), False)
p.set_cmap(('bands', 'B04_20'), 'B-W LINEAR')
p.show()

AttributeError: 'GeoTiffWindowDataset' object has no attribute 'resolution'

## Rasterio image up/down sampling

In [ ]:
s2_file = ('C:/Users/arevi/OneDrive/YT_GITHUB_v2/TEST_DATASETS/test_resampling_v2.tif')

In [ ]:
with rasterio.open(s2_file) as dataset:
    original_image = dataset.read()

In [ ]:
fig, ax1 = plt.subplots(1,1, figsize=(10, 10))
ax1.imshow(original_image[1,:,:], 'RdYlGn', interpolation='none')

In [ ]:
upscale_factor = 0.5

In [ ]:
with rasterio.open(s2_file) as dataset:

    # resample data to target shape
    resampled = dataset.read(
        out_shape=(
            dataset.count,
            int(dataset.height * upscale_factor),
            int(dataset.width * upscale_factor)
        ),
        resampling=Resampling.bilinear
    )

    # scale image transform
    transform = dataset.transform * dataset.transform.scale(
        (dataset.width / resampled.shape[-1]),
        (dataset.height / resampled.shape[-2])
    )

In [ ]:
original_image.shape

In [ ]:
resampled.shape

In [ ]:
fig, ax1 = plt.subplots(1,1, figsize=(10, 10))
ax1.imshow(resampled[1,:,:], 'RdYlGn', interpolation='none')

## Rasterio reprojecting image

In [ ]:
# destination geographical coordinate system
dst_crs = 'EPSG:4326'

In [ ]:
with rasterio.open(s2_file) as dataset:
    original_image = dataset.read(1)
    print(dataset.crs)

In [ ]:
with rasterio.open(s2_file) as src:
    print(src.shape)
    print(src.dtypes)
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds)
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })

    reproject_array = np.zeros((224,472),dtypes=np.uint8)
    
    with rasterio.open('C:/Users/arevi/OneDrive/YT_GITHUB_v2/TEST_DATASETS/test_crs.tif', 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest)
        print(dst.shape)

In [ ]:
np.zeros((224,472))

In [ ]:
num = 20.0

In [ ]:
num2=round(num)

In [ ]:
num2

In [ ]:
data = {"filename": 'FILENAME', "resolution": 'RESOLUTION'}